In [1]:
import numpy as np
import pandas as pd

from psql_functions import execQuery
from miss_data import add_missing_dates, add_missing_counts
from sample_range_query import load_range_queries_n_split

from flat_olh import OLH_flat

import re
import os
owd = os.getcwd()

In [2]:
os.chdir(owd)

In [3]:
param_dic = {
    "host"      : "localhost",
    "database"  : "bachelorBesoeg2014",
    "user"      : "postgres",
    "password"  : "password",
    "port"      : "5432"
}

query = """select time_ from _775147;"""
result = execQuery(param_dic, query)
dates = [(date[0]) for date in result]

query = """select count_ from _775147;"""
result = execQuery(param_dic, query)

counts = [(count[0]) for count in result]

all_dates = add_missing_dates(dates)
all_counts =  add_missing_counts(counts, dates, all_dates)


Executed query and closed connection.
Executed query and closed connection.


In [4]:
n_data_structures = 2

n = np.array([32,128,256,512,1024,2048])
degrees = np.array([2,3,4])

flat_r_32 =  np.array([2, 4, 8, 12, 16, 20, 24])
flat_r_128 =  np.array([20, 40, 50, 60, 70, 80, 90])
flat_r_256 =  np.array([40, 60, 80, 100, 140, 200, 220])
flat_r_512 =  np.array([100, 150, 200, 250, 300, 400, 450])
flat_r_1024 =  np.array([200, 300, 400, 500, 600, 800, 900])
flat_r_2048 =  np.array([600, 800, 1000, 1250, 1500, 1700, 1800])

rs =  [flat_r_32, flat_r_128, flat_r_256, flat_r_512, flat_r_1024, flat_r_2048]

In [5]:
def make_query_lookup_dict_n_degree(n, degrees):
    n_dict = dict.fromkeys(n)
    degree_dicts = []
    for i in range(0,len(n)):
        degree_dict = dict.fromkeys(degrees)
        degree_dicts.append(degree_dict)

    for idx, n_key in enumerate(n_dict):
            n_dict[n_key] = degree_dicts[idx]
    
    return n_dict

def fill_up_query_lookup_dict_n_degree(n, degrees, files):
    dict_ = make_query_lookup_dict_n_degree(n, degrees)
    for idx, f in enumerate(files):
        here_n = re.search(r'N=\d+', f).group(0)
        here_n = here_n.split("N=")[1]
        here_degree = re.search(r'B=\d+', f).group(0) 
        here_degree = here_degree.split("B=")[1]
        
        querries = load_range_queries_n_split(f, n_data_structures)
        querries = [item for sublist in querries for item in sublist]
        dict_[int(here_n)][int(here_degree)] = querries.copy()
    return dict_

In [6]:
os.chdir(owd)
print(os.getcwd())
os.chdir(owd+'/'+'range_queries/local_hh')
print(os.getcwd())
hh_files = [f for f in os.listdir('.') if re.match(r'.*\.csv', f)]
print(hh_files)
queries_n_degree = fill_up_query_lookup_dict_n_degree(n, degrees, hh_files)

C:\Users\Jonas Friis\Documents\Universitet\Bachelors-Thesis\src
C:\Users\Jonas Friis\Documents\Universitet\Bachelors-Thesis\src\range_queries\local_hh
['hh_N=1024_B=2_r=450.csv', 'hh_N=1024_B=3_r=296.csv', 'hh_N=1024_B=3_r=500.csv', 'hh_N=1024_B=4_r=300.csv', 'hh_N=2048_B=2_r=512.csv', 'hh_N=2048_B=3_r=400.csv', 'hh_N=2048_B=3_r=800.csv', 'hh_N=2048_B=4_r=350.csv', 'hh_N=256_B=2_r=44.csv', 'hh_N=256_B=3_r=217.csv', 'hh_N=256_B=3_r=230.csv', 'hh_N=256_B=4_r=200.csv', 'hh_N=512_B=2_r=400.csv', 'hh_N=512_B=3_r=217.csv', 'hh_N=512_B=3_r=300.csv', 'hh_N=512_B=4_r=300.csv']


In [7]:
queries_n_degree

{32: {2: None, 3: None, 4: None},
 128: {2: None, 3: None, 4: None},
 256: {2: ['2014-04-14, 2014-05-27',
   '2014-06-30, 2014-08-12',
   '2014-04-04, 2014-05-17',
   '2014-01-16, 2014-02-28',
   '2014-04-18, 2014-05-31',
   '2014-03-14, 2014-04-26',
   '2014-07-09, 2014-08-21',
   '2014-01-22, 2014-03-06',
   '2014-04-14, 2014-05-27',
   '2014-05-03, 2014-06-15',
   '2014-07-31, 2014-09-12',
   '2014-03-17, 2014-04-29',
   '2014-07-23, 2014-09-04',
   '2014-03-30, 2014-05-12',
   '2014-04-28, 2014-06-10',
   '2014-04-11, 2014-05-24',
   '2014-04-15, 2014-05-28',
   '2014-06-02, 2014-07-15',
   '2014-05-12, 2014-06-24',
   '2014-05-31, 2014-07-13',
   '2014-02-23, 2014-04-07',
   '2014-01-03, 2014-02-15',
   '2014-03-30, 2014-05-12',
   '2014-06-08, 2014-07-21',
   '2014-02-08, 2014-03-23',
   '2014-05-11, 2014-06-23',
   '2014-07-12, 2014-08-24',
   '2014-07-08, 2014-08-20',
   '2014-01-22, 2014-03-06',
   '2014-06-11, 2014-07-24',
   '2014-07-24, 2014-09-05',
   '2014-02-28, 2014-04-

In [8]:
def fill_up_query_lookup_dict_n(n, files):
    dict_ = dict.fromkeys(n)
    for idx, f in enumerate(files):
        here_n = re.search(r'N=\d+', f).group(0)
        here_n = here_n.split("N=")[1]
        
        querries = load_range_queries_n_split(f, n_data_structures)
        dict_[int(here_n)] = querries.copy()
    return dict_

In [9]:
os.chdir(owd)
print(os.getcwd())

os.chdir(os.getcwd()+'/'+'range_queries/flat')
print(os.getcwd())
flat_files = [f for f in os.listdir('.') if re.match(r'.*\.csv', f)]
print(flat_files)
flat_dict = fill_up_query_lookup_dict_n(n, flat_files)
print(flat_dict)

C:\Users\Jonas Friis\Documents\Universitet\Bachelors-Thesis\src
C:\Users\Jonas Friis\Documents\Universitet\Bachelors-Thesis\src\range_queries\flat
['flat_N=1024_r=44.csv', 'flat_N=128_r=9.csv', 'flat_N=2048_r=64.csv', 'flat_N=256_r=24.csv', 'flat_N=32_r=6.csv', 'flat_N=512_r=35.csv']
{32: [array(['2014-01-07, 2014-01-12', '2014-01-04, 2014-01-09',
       '2014-01-14, 2014-01-19', ..., '2014-01-24, 2014-01-29',
       '2014-01-26, 2014-01-31', '2014-01-02, 2014-01-07'], dtype=object), array(['2014-01-14, 2014-01-19', '2014-01-09, 2014-01-14',
       '2014-01-13, 2014-01-18', ..., '2014-01-27, 2014-02-01',
       '2014-01-19, 2014-01-24', '2014-01-24, 2014-01-29'], dtype=object)], 128: [array(['2014-01-14, 2014-01-22', '2014-01-29, 2014-02-06',
       '2014-01-17, 2014-01-25', ..., '2014-03-05, 2014-03-13',
       '2014-02-08, 2014-02-16', '2014-04-24, 2014-05-02'], dtype=object), array(['2014-03-07, 2014-03-15', '2014-01-13, 2014-01-21',
       '2014-03-25, 2014-04-02', ..., '2014-04-23

In [10]:
def make_query_lookup_dict_n_r(n, rs):
    n_dict = dict.fromkeys(n)
    r_dicts = []
    for i in range(0,len(n)):
        r_dict = dict.fromkeys(rs[i])
        r_dicts.append(r_dict)

    for idx, n_key in enumerate(n_dict):
            n_dict[n_key] = r_dicts[idx]
    
    return n_dict

def fill_up_query_lookup_dict_n_r(n, rs, files):
    dict_ = make_query_lookup_dict_n_r(n, rs)
    for idx, f in enumerate(files):
        here_n = re.search(r'N=\d+', f).group(0)
        here_n = here_n.split("N=")[1]
        here_r = re.search(r'r=\d+', f).group(0) 
        here_r = here_r.split("r=")[1]
        
        querries = load_range_queries_n_split(f, n_data_structures)
        dict_[int(here_n)][int(here_r)] = querries.copy()
    return dict_

In [11]:
os.chdir(owd)
print(os.getcwd())
os.chdir(os.getcwd()+'/'+'range_queries/flat_varying_r')
print(os.getcwd())
files = [f for f in os.listdir('.') if re.match(r'.*\.csv', f)]
print(files)
flat_n_r_dict = fill_up_query_lookup_dict_n_r(n, rs, files)

C:\Users\Jonas Friis\Documents\Universitet\Bachelors-Thesis\src
C:\Users\Jonas Friis\Documents\Universitet\Bachelors-Thesis\src\range_queries\flat_varying_r
['flat_N=1024_r=200.csv', 'flat_N=1024_r=300.csv', 'flat_N=1024_r=400.csv', 'flat_N=1024_r=500.csv', 'flat_N=1024_r=600.csv', 'flat_N=1024_r=800.csv', 'flat_N=1024_r=900.csv', 'flat_N=128_r=20.csv', 'flat_N=128_r=40.csv', 'flat_N=128_r=50.csv', 'flat_N=128_r=60.csv', 'flat_N=128_r=70.csv', 'flat_N=128_r=80.csv', 'flat_N=128_r=90.csv', 'flat_N=2048_r=1000.csv', 'flat_N=2048_r=1250.csv', 'flat_N=2048_r=1500.csv', 'flat_N=2048_r=1700.csv', 'flat_N=2048_r=1800.csv', 'flat_N=2048_r=600.csv', 'flat_N=2048_r=800.csv', 'flat_N=256_r=100.csv', 'flat_N=256_r=140.csv', 'flat_N=256_r=200.csv', 'flat_N=256_r=220.csv', 'flat_N=256_r=40.csv', 'flat_N=256_r=60.csv', 'flat_N=256_r=80.csv', 'flat_N=32_r=12.csv', 'flat_N=32_r=16.csv', 'flat_N=32_r=2.csv', 'flat_N=32_r=20.csv', 'flat_N=32_r=24.csv', 'flat_N=32_r=4.csv', 'flat_N=32_r=8.csv', 'flat_N=51

In [12]:
for n_key in flat_n_r_dict:
    print(flat_n_r_dict[n_key].keys())
    for r in flat_n_r_dict[n_key]:
        print(len(flat_n_r_dict[n_key][r]))


dict_keys([2, 4, 8, 12, 16, 20, 24])
2
2
2
2
2
2
2
dict_keys([20, 40, 50, 60, 70, 80, 90])
2
2
2
2
2
2
2
dict_keys([40, 60, 80, 100, 140, 200, 220])
2
2
2
2
2
2
2
dict_keys([100, 150, 200, 250, 300, 400, 450])
2
2
2
2
2
2
2
dict_keys([200, 300, 400, 500, 600, 800, 900])
2
2
2
2
2
2
2
dict_keys([600, 800, 1000, 1250, 1500, 1700, 1800])
2
2
2
2
2
2
2


In [13]:
print(flat_n_r_dict[1024].keys())
print(flat_n_r_dict[1024])
print(len(flat_n_r_dict[1024]))
print(len(flat_n_r_dict[1024][200]))
print(len(flat_n_r_dict[1024][200][0]))
#print(len(flat_n_r_dict[1024][200]))

dict_keys([200, 300, 400, 500, 600, 800, 900])
{200: [array(['2014-02-06, 2014-08-24', '2014-05-11, 2014-11-26',
       '2014-12-20, 2015-07-07', ..., '2014-04-19, 2014-11-04',
       '2015-06-06, 2015-12-22', '2015-04-16, 2015-11-01'], dtype=object), array(['2015-11-12, 2016-05-29', '2014-08-08, 2015-02-23',
       '2015-11-02, 2016-05-19', ..., '2016-02-03, 2016-08-20',
       '2016-03-31, 2016-10-16', '2015-01-29, 2015-08-16'], dtype=object)], 300: [array(['2015-03-12, 2016-01-05', '2015-12-16, 2016-10-10',
       '2015-10-26, 2016-08-20', ..., '2015-07-13, 2016-05-07',
       '2014-07-08, 2015-05-03', '2014-11-27, 2015-09-22'], dtype=object), array(['2014-04-25, 2015-02-18', '2015-07-17, 2016-05-11',
       '2014-11-15, 2015-09-10', ..., '2015-06-19, 2016-04-13',
       '2015-09-26, 2016-07-21', '2015-10-26, 2016-08-20'], dtype=object)], 400: [array(['2015-08-25, 2016-09-27', '2014-01-09, 2015-02-12',
       '2014-01-18, 2015-02-21', ..., '2014-06-08, 2015-07-12',
       '2015-07-2

In [14]:
def model_answers(model, query):
    estimation = model.answer(query)
    correct = model.real_answer(query)  
    return estimation, correct

def get_answers_for_variable(n_r, step_size, all_answers):
    """
    Setup of the datastructere
    Parameters:
    n_r (int): Which index/number of the variable values
    step_size (int): How many variable values there is 
    all_answers (Array): All answers for all the differnt variable values
    Returns:
    The answers for this specific variable
    """
    return all_answers[n_r::step_size]

In [15]:
epsilons = np.array([2, 1.4, 1.2, 1, 0.8, 0.6, 0.5, 0.3])
n = np.array([32, 128, 256, 512, 1024, 2048])
degrees = np.array([2, 3, 4])

os.chdir(owd)

for e in epsilons:
    print(f'Epsilon = {e}')
    for N in n:
        print(f'N = {N}')
        
        all_r_queries_est = []
        all_r_queries_cor = []
        
        all_b_queries_est = []
        all_b_queries_cor = []
        
        all_flat_queries_est = []
        all_flat_queries_cor = []
        
        
        varying_r_queries = flat_n_r_dict[N]
        hh_queries = queries_n_degree[N]
        flat_queries = flat_dict[N]
        #print(varying_r_queries.keys())
        
        for i in range(0,n_data_structures):
            model = OLH_flat(e, all_dates[:N], all_counts[:N])

            for r_key in varying_r_queries:
                #print(r_key)
                r_queries = varying_r_queries[r_key]
                #print(r_queries[i])
                r_est = []
                r_cor = []
                for query in r_queries[i]:
                    dates_split = query.split(', ')
                    dates = tuple(dates_split)

                    est, cor = model_answers(model, dates)
                    
                    r_est.append(est)
                    r_cor.append(cor)
                
                all_r_queries_est.append(r_est)
                all_r_queries_cor.append(r_cor)
            
            for b_key in hh_queries:
                b_queries = hh_queries
                
                b_est = []
                b_cor = []
                for query in b_queries[i]:
                    dates_split = query.split(', ')
                    dates = tuple(dates_split)

                    est, cor = model_answers(model, dates)
                    
                    b_est.append(est)
                    b_cor.append(cor)
                
                all_b_queries_est.append(b_est)
                all_b_queries_cor.append(b_cor)
            
            for query in flat_queries[i]:
                dates_split = query.split(', ')
                dates = tuple(dates_split)

                est, cor = model_answers(model, dates)
                    
                all_flat_queries_est.append(est)
                all_flat_queries_cor.append(cor)
            
        """      
        for idx, r_key in enumerate(varying_r_queries):
            csv_name_r_est = 'results/sample_querys/flat_varying_r/' + f'est_e={e}_N={N}_r={r_key}.csv'
            csv_name_r_cor = 'results/sample_querys/flat_varying_r/' + f'cor_e={e}_N={N}_r={r_key}.csv'
            
            r_answers = get_answers_for_r(idx, len(rs[0]), all_r_queries_est)
            r_answers = [item for sublist in r_answers for item in sublist]
            
            r_correct = get_answers_for_r(idx, len(rs[0]), all_r_queries_cor)
            r_correct = [item for sublist in r_correct for item in sublist]
            
            np.savetxt(csv_name_r_est, r_answers, delimiter=',')
            np.savetxt(csv_name_r_cor, r_correct, delimiter=',')
        """  
        
        for idx, b_key in enumerate(hh_queries):
            csv_name_est_hh = 'results/sample_querys/flat/' + f'est_hh_queries_e={e}_N={N}_B={degree}.csv'
            csv_name_cor_hh = 'results/sample_querys/flat/' + f'cor_hh_queries_e={e}_N={N}_B={degree}.csv'
            
            b_answers = get_answers_for_variable(idx, len(degrees), all_r_queries_est)
            b_answers = [item for sublist in b_answers for item in sublist]
            
            b_correct = get_answers_for_variable(idx, len(degrees), all_r_queries_cor)
            b_correct = [item for sublist in b_correct for item in sublist]
            
            np.savetxt(csv_name_est_hh, b_answers, delimiter=',')
            np.savetxt(csv_name_cor_hh, b_correct, delimiter=',')
            
        csv_name_est_flat = 'results/sample_querys/flat/' + f'est_flat_queries_e={e}_N={N}.csv'
        csv_name_cor_flat = 'results/sample_querys/flat/' + f'cor_flat_queries_e={e}_N={N}.csv'
        np.savetxt(csv_name_est_flat, all_flat_queries_est, delimiter=',')
        np.savetxt(csv_name_cor_flat, all_flat_queries_cor, delimiter=',')
        """           
        np.savetxt(csv_name_est_flat, answers_flat, delimiter=',')
        np.savetxt(csv_name_cor_flat, correct_answers_flat, delimiter=',')

        np.savetxt(csv_name_est_hh, answers_hh, delimiter=',')
        np.savetxt(csv_name_cor_hh, correct_answers_hh, delimiter=',')
        
        """ 

Epsilon = 2.0
N = 32


KeyError: 0

In [ ]:
print(os.getcwd())


In [ ]:
a = np.arange(0,30)

In [ ]:
all_r_queries_est

In [ ]:
print(len(all_r_queries_cor[0]))

In [ ]:
get_answers_for_variable(0,3,a)